# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from ipywidgets.embed import embed_minimal_html

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
csv_file = "output_data/cities.csv"
city_data = pd.read_csv(csv_file)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [ ]:
# Bring in location data pairs, humidity data, define map and layer, and add layer to map
locations = city_data[["Lat", "Lng"]] 
humid_data = city_data["Humidity"]

fig = gmaps.figure(center=(20, 12.513),zoom_level=1.5)

humidity_layer = gmaps.heatmap_layer(locations, weights=humid_data, 
                                 dissipating=False, max_intensity=humid_data.max(),
                                 point_radius = 1)

fig.add_layer(humidity_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Drop any rows with null values
city_data = city_data.dropna(how='all')

# Filter cities to fit temperate weather conditions
ideal_cities = city_data.loc[(city_data['Max Temp'] >= 70) & (city_data['Max Temp'] < 80)]
ideal_cities = ideal_cities.loc[city_data['Wind Speed'] < 10]
ideal_cities = ideal_cities.loc[city_data['Cloudiness'] == 0]

### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Create dataframe and empty column for Hotel Name
hotel_df = pd.DataFrame(ideal_cities[['City', 'Lat', 'Lng', 'Country']])
hotel_df['Hotel Name'] = ''

In [ ]:
# Set up the base URL and initial parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "types": "lodging",
    "radius": 5000,
    "key": g_key,
    }

# Loop through dataframe and add nearby hotel result
for index, row in hotel_df.iterrows():

    # Get Latitude/Longitude
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add Location to params dict
    params['location'] = f"{latitude}, {longitude}"
    
    # Assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response['results']

    try:
        # Update hotel DataFrame with nearby Hotel Name, Latitude, and Longitude
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = results[0]['geometry']['location']['lng']

    except (KeyError, IndexError):
        print(f"There are no lodgings within 5K of {row['City']}.")
        pass

In [ ]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig